In [54]:

import requests
import numpy as np
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

## Get headers from data definition:

In [103]:
def get_headers(url):
    response = requests.get(
    url=url,
    headers={
        "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding" : "gzip, deflate, br",
        "Accept-Language" : "en-US,en;q=0.9",
        "Cache-Control" : "no-cache",
        "Cookie" : "modal-scrim-ask-id-73=true; _opensecrets_session=sMwuf9Rpm31Z2UvYVcyAi42nViIBSIFrTGiVZuWqwwPE8zAYjaKcVXX2jhAJIEzYwkWfOI9ysif0WJJCD5xaudRlQuulzLewc3%2FNEYNb3y32JMb%2FASQKSAAYigXFcyLN6cFydGswxDLj5k61N5%2FFksUICbAJYn52DojezYTY9lHv%2Fq7Mw%2BQFo%2BWjMyPxlfJoFTo4jvIZyqSlCFG5Z%2BKXdO25fmPAW5NzpbYerwFxzlXJd9W%2BTNOAUAUc5v9rqfMBfeY0hTn2%2BY1x45uot1XXcYQRApeFo4gFr1DfcA%3D%3D--Ea3B%2Bqd%2FQJWzbdYJ--Xo4X4q7KuDYNef2RkhgLyw%3D%3D",
        "Pragma" : "no-cache",
        "Referer": "https:/www.opensecrets.org/members-of-congress",
        "Sec-Ch-Ua" : '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        "Sec-Ch-Ua-Mobile" : "?0",
        "Sec-Ch-Ua-Platform" : "macOS",
        "Sec-Fetch-Dest" : "document",
        "Sec-Fetch-Mode" : "navigate",
        "Sec-Fetch-Site" : "same-origin",
        "Sec-Fetch-User" : "?1",
        "Upgrade-Insecure-Requests" : "1",
        "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
    })

    headers = BeautifulSoup(response.content)

    return pd.read_html(StringIO(str(headers.findAll("table")[0])),header=0)[0]

    

## Build DF with data from 1990 using headers from before 2015:

In [111]:

# Get headers from webscrape:
cand_header_df = get_headers("https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20Candidates%20Data.htm")

# Load data with columns from scraped data dict:
df = pd.read_csv("cands90.txt", names=list(cand_header_df["Field"].apply(lambda x: x.lower()).values))
# Clean fields:
df = df.apply(lambda x: str(x).replace("|","")).replace(['    ', ' '], np.nan) 
# Only include current candidates
df = df[df["currcand"] == "Y"].reindex()
# Only include house representatives:
s_mask = df["distidrunfor"].apply(lambda x: True if "S1" not in x or "S2" not in x else False)
df = df[s_mask]

df

KeyError: False

In [128]:
def load_txt_data(filename, dictionary_url):
    # Get headers from webscrape:
    header_df = get_headers(dictionary_url)

    # Load data with columns from scraped data dict:
    df = pd.read_csv(filename, names=list(header_df["Field"].apply(lambda x: x.lower()).values))
    # Clean fields:
    df = df.apply(lambda x: x.astype(str).str.replace("|","")).replace(['    ', ' '], np.nan) 

    return df 


In [152]:
pacs_df = load_txt_data("pacs90.txt","https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20for%20PAC%20to%20Cands%20Data.htm")
# Include only direct condtibutions to candidates:
pacs_df = pacs_df[pacs_df["di"]=='D']
pacs_df

,cycle,fecrecno,pacid,cid,amount,date,realcode,type,di,feccandid
0,1990,3062020110011107576,C00198614,N00006246,250,04/07/1989,Z1100,24K,D,H8WY00106
1,1990,3062020110009889492,C00105981,N00004643,5000,02/01/1989,F2100,24K,D,S8MT00010
2,1990,3062020110011174239,C00034330,N00004781,500,03/27/1989,M7200,24K,D,H6IL14095
3,1990,3062020110011715035,C00027532,N00002519,1500,04/20/1989,LT500,24K,D,H4GA07135
4,1990,3062020110010294322,C00183087,N00003038,1000,02/01/1989,K2000,24K,D,S8AL00050
...,...,...,...,...,...,...,...,...,...,...
189288,1990,3062020110011315778,C00114017,N00000352,250,08/03/1990,B5100,24K,D,H0RI02105
189289,1990,3062020110012602392,C00235846,N00002260,1000,09/24/1990,F3300,24K,D,H6NC04037
189290,1990,3062020110011095144,C00097725,N00001093,5000,02/13/1990,F3100,24K,D,H0NY16010
189291,1990,3062020110010959373,C00035709,N00007999,1000,05/26/1989,E1140,24K,D,H6AK00045


In [153]:

print("num contributions:", len(pacs_df))
print("num candidates:", len(pacs_df["cid"].unique()))
print("num pacs:", len(pacs_df["pacid"].unique()))

num contributions: 189293
num candidates: 1397
num pacs: 3614


In [161]:
pacs_df = pacs_df.groupby(["cid","pacid"])["amount"].sum().reset_index()

for cid in pacs_df["cid"].unique():
    pacs_df.loc[pacs_df["cid"]==cid, "pac_count"] = len(pacs_df[pacs_df["cid"]==cid])

pacs_df

,cid,pacid,amount,pac_count
0,N00000003,C00000422,350500,219.0
1,N00000003,C00000729,4500500,219.0
2,N00000003,C00000885,500,219.0
3,N00000003,C00000901,10005001000,219.0
4,N00000003,C00000935,215233,219.0
...,...,...,...,...
109079,N00028986,C00033969,300,6.0
109080,N00028986,C00077321,500,6.0
109081,N00028986,C00094870,64357,6.0
109082,N00028986,C00122101,10003000,6.0
